# Prjeto 2
dependecies:
pip install
>Tkinter
>
>pyTorch
>
>tqdm
>
>ipywidgets

In [1]:
import numpy as np
#Interface
from tkinter import *
from tkinter import ttk
import time
#Comunication
import socket
import re
import traceback

##### necessary values for tkinter:

In [2]:
# values necessary to construct interface
canva_size=700
color1='black'
color2='white'
default=0
#grid_size=args['size'] deixou de ser preciso, pois ou é 4 ou 6 dependento do size do ataxx 
go_offset=45
EAST=[1,0]
SOUTH=[0,1]
WEST=[-1,0]
NORTH=[0,-1]

# configurações do jogo


In [3]:
#  __HERE_YOU_CAN_EDIT_YOU_GAME_PERFERENCES__
#  *   Scroll EVERYTHING DOWN TO EDIT   *
#  PLEASE, EDIT ONLY WHERE IS MARKED WITH:  #&_EDIT_HERE_&#

comunication = {
    'on': True,             #if comunication is on, , run server.py first and then the mains
    'off': False,
    'port': 12401,
    'server_host':'localhost',
    'client_host':'localhost',
}

#ATAXX
# IF you want to add non playable blocks, use number 3
board4 = {
    'size': 4,
    'board': np.array([[1, 0, 0, -1],          #&_EDIT_HERE_&# -> edit initial state as you wish 
                        [0, 0, 0, 0],
                        [0, 0, 0, 0],
                        [-1, 0, 0, 1]])
}
board5 = {
    'size': 5,
    'board': np.array([[1, 0, 0, 0, -1],    #&_EDIT_HERE_&# -> edit initial state as you wish
                        [0, 0, 0, 0, 0],
                        [0, 0, 3, 0, 0],
                        [0, 0, 0, 0, 0],
                        [-1, 0, 0, 0, 1]])
}
board6 = {
    'size': 6,
    'board': np.array([[1, 0, 0, 0, 0, -1],    #&_EDIT_HERE_&# -> edit initial state as you wish
                        [0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 3, 0, 0],
                        [0, 0, 3, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0],
                        [-1, 0, 0, 0, 0, 1]])
}

#GO
board7 = {
    'size': 7,
    'board': np.array([[0, 0, 0, 0, 0, 0, 0],    #&_EDIT_HERE_&# -> edit initial state as you wish
                        [0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0]])
}

board9 = {
    'size': 9,
    'board': np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0],    #&_EDIT_HERE_&# -> edit initial state as you wish
                        [0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 0, 0]])
}


game_mode = {
    'agent_vs_agent': 3,
    'agent_vs_human': 2,
    'human_vs_human': 1,
}

jogar = {
    'Ataxx': 'A',
    'Go': 'G'
}

#GAME_PREFERENCES      <edit_here>                         #&_EDIT_HERE_&# :
configurations = {
    #with comunication, edit below:
    'server_game': "G7x7",                                  #"A6x6" "A4x4" "G7x7" "G9x9"
    'comunication': comunication['off'],                    #'off'/'on'. -> if 'off' edit below: 
    #without comunication, set it off and edit below:
    'size': 7,                                              #4/6 -> ataxx
    'game_mode': game_mode['agent_vs_human'],               #['agent_vs_agent']/['agent_vs_human']/['human_vs_human'],
    #common to both:
    'time_between_moves': 10,                             #time_between_moves without comunication in miliseconds
    'jogo': jogar['Go']                                  #[Ataxx]/[Go], por enquanto só ataxx
}

#...check update...

Both Ataxx and Go are built in a way so that the state, player and verification features are external to the class in order to be shared in comunication, montecarlo different braches and interface.

Both Ataxx and Go have an interface.

Both can be played:
> pc vs pc with and without comunication
>
> pc vs human without comunication
>
> human vs human without comunication

# Attax

In [9]:
class Ataxx:
    def __init__(self,size):
        self.size = size
        self.action_size = self.act_size()

    def make_s0(self):
        #used only for testing and debugging
        matrix = np.zeros((self.size, self.size), dtype=int)
        matrix[0][0] = 1 #white
        matrix[self.size-1][self.size-1] = 1 #white
        matrix[self.size-1][0] = -1 #black
        matrix[0][self.size-1] = -1 #black
        return matrix

    def place(self,x,y,state,player):
        state[x][y] = player
        return state
      
    def remove(self,x,y,state):
        state[x][y] = 0
        return state
    
    def get_state(self):
        return self.state
    
    def count(self,player,state): 
        x, _ = np.where(state == player)
        return len(x)
    
    def change_turn(self,player):
        return -player

    def get_color(self,player):
        if player == 1:
            return 'white'
        else:
            return 'black'
    
    def distance(self,x,y,movex,movey):                 
        difx=abs(x-movex)
        dify=abs(y-movey)
        return max(difx,dify)
        
    def isjump(self,x,y,movex,movey):
        return self.distance(x,y,movex,movey)==2    
    
    def isfree(self,x,y,state):                               
        return state[x][y]==0
    
    def infect_neighbours(self,x,y,state,player):
        infection_space = self.radius_space(x,y,1)
        for (line,col) in infection_space:
            if state[line][col] != 0 and state[line][col] != 3:
                self.place(line,col,state,player)  
    
    def radius_space(self,x,y,radius): 
        # All the possible moves for a pick (x,y)
        # The radius is either 1 or 2
        radius_space = []   
        # Check radius space
        bottom = x+radius
        left = y-radius
        top = x-radius
        right  = y+radius
        # Check if the radius space is inside the board
        while left<0:
            left+=1
        while top<0:
            top+=1
        while right > (self.size-1):
            right-=1
        while bottom > (self.size-1):
            bottom-=1
        # From radius space select all positions (empty included)
        for line in range(top,bottom+1):           
            for col in range(left, right+1):        
                radius_space.append((line,col)) 
        return radius_space  
    
    def possible_picks(self,state, player):                           
        # All the possible picks for the player
        x, y = np.where(state == player)
        return list(zip(x,y))
    
    def picked_action_space(self,x,y,state):
        # Action space from picked (x,y) (radius 2 and 1)
        picked_action_space = []
        aux = self.radius_space(x,y,2)
        for (line,col) in aux:
            if self.isfree(line,col,state):
                picked_action_space.append((line,col))
        return picked_action_space  # Returns list of tuples 

    def action_space(self,state,player):
        #Returns action space for the current state and player
        all_actions = []
        picks = self.possible_picks(state,player)
        for (x,y) in picks:
            for (movex,movey) in self.picked_action_space(x,y,state):
                if movex is not None and movey is not None:
                    action = (x,y,movex,movey)      
                    all_actions.append(action)
        return all_actions 
    
    def play_move(self,action,state,player):
        #the move comes already verified
        self.place(action[2], action[3],state,player)
        #if jump remove father piece
        if self.isjump(action[0], action[1], action[2], action[3]):
            self.remove(action[0], action[1],state)
        self.infect_neighbours(action[2], action[3],state,player)
        return state.copy() #copy just in case
    
    def verify_move(self,action, state, player):
        return action in self.action_space(state,player)
    
    def get_random_move(self,state,player):
        #For playing Randomly
        state = state.copy()#just in case
        action_space = self.action_space(state,player)
        action = action_space[np.random.randint(0,len(action_space))]
        return action
            
    def add_to_lastnine(self,state,last_nine):  
        #receives last nine and updates it
        last = last_nine.copy()
        novo = state.copy()
        if (len(last)==9):
            last.pop(0)
            last.append(novo)
        else:
            last.append(novo)
        return last.copy()
    
    def is_threefold_repetition_rule(self,last_nine):
        dic={}
        if  last_nine and len(last_nine)==9:
            for i in last_nine:
                #trasformar em tuplo(hashable) para poder ser chave do dicionário
                tuplo = tuple(map(tuple, i))
                if tuplo in dic.keys():
                    dic[tuplo] += 1
                else:
                    dic[tuplo] = 1
            for tuplo in dic.keys():
                if dic[tuplo]>=3:
                    return True
        return False
    
    def isfull(self, state):
        x, _ =np.where(state == 0)
        if len(x)>0:
            return False
        return True
    
    def is_stuck(self, state, player):
        return ((len(self.action_space(state, player)) == 0) and (not self.isfull(state)))
    
    def is_finished(self, state, player, last_nine):
        if self.isfull(state) or self.is_stuck(state,player) or self.is_threefold_repetition_rule(last_nine):
            return True
        return False
    
    def winner(self, state, player, last_nine):
        #if_Full
        if self.isfull(state):
            print("Game is Full!")
            if (self.count(1,state) - self.count(-1,state) > 0):
                return 1
            elif (self.count(1,state) - self.count(-1,state) < 0):
                return -1
            else:                
                return 0
            
        #if_Stuck
        elif self.is_stuck(state, player):
            print(player, "is stuck!")
            self_count = self.count(state,player)
            opponent_possible_count = self.size*self.size - self_count
            if self_count > opponent_possible_count:
                return player
            elif self_count < opponent_possible_count:
                return -player
            else:
                return 0
        
        elif self.is_threefold_repetition_rule(last_nine):
            # não queremos que o modelo aprenda a ganhar por esta regra!
            print("Threefold Repetition Rule!")
            return 0
    
        return 0    #other non imagined situation
            


    #______________________Functions_for_Alpha_Zero____________________#

    def get_value_and_terminated(self,state,player):
        if self.isfull(state):
            if (self.count(1,state) != self.count(-1,state)):
             return 1, True      # White or Black wins
            else:
                return 0, True   # Draw
        if self.is_stuck(state,player):
            if(player == 1):
                color_turn = 'white'
            else:
                color_turn = 'black'
            self_count = self.count(player,state)
            opponent_possible_count = self.size*self.size - self_count
            if(self_count != opponent_possible_count):
                return 1, True    # White or Black wins  
            else:
                return 0, True  # Draw
        return 0, False
    
    def get_opponent_value(self, value):
        return -value
    
    def change_perspective(self, state):
        # Changes the state so that 1's become -1's and vice versa.
        # Useful for the algorithm
        return state*-1 
    
    # Some functions useful for the neural network
    def get_encoded_state(self, state):
        encoded_state = np.stack(
            (state == -1, state == 0, state == 1)
        ).astype(np.float32)
        
        # AlphaParallel modifcation
        if len(state.shape) == 3:
            encoded_state = np.swapaxes(encoded_state, 0, 1)
        
        return encoded_state
    
    def action_dict(self):
        # Turns all possible actions into a dictionary 
        actions = self.get_all_actions()
        dic = {}
        for action in actions:
            dic[action] = 0.0   
        return dic
    
    def get_all_actions(self):
        # Gets all the possible actions 
        all_actions = []
        empty_board = np.zeros((self.size, self.size), dtype=int)
        # All possible picks
        x,y = np.where(empty_board == 0)
        picks = list(zip(x, y))
        for (x,y) in picks:
            for (movex,movey) in self.picked_action_space(x,y,empty_board):
                if movex is not None and movey is not None:
                    action = (x,y,movex,movey)      
                    all_actions.append(action)
        return all_actions 
    
    def act_size(self):
        return len(self.get_all_actions())



# interface Attax

In [5]:
class interfaceAtaxx():

    def __init__(self, size, game, initial_state, type_game, time_between_moves, client = None, ag = None):

        self.size = size
        self.game = game
        self.state = initial_state
        self.time_between_moves = time_between_moves    #only without comunication

        if client:
            self.comunication = True
            self.client = client
            self.ag=ag                                  #ag1=1 ag2=-1
            self.first=True
            type_game = 3                               #override, pc contra pc obrigatoriamente

        #first click
        self.click=True
        self.click_last_nine = [initial_state.copy()]
        self.human_player = 1

        self.root, self.canva = self.make_clean_canvas(canva_size)
        self.draw_default(self.canva)

        if(type_game==1):   
            self.root.bind("<Button-1>", self.human_x_human_off)
        elif(type_game==2):
            self.root.bind("<Button-1>", self.human_vs_pc_off_random)
        elif(type_game==3):
            if client: 
                self.pc_x_pc_on_random(initial_state)
            else: #ofline
                self.pc_x_pc_off_random(initial_state)

        self.root.title("Ataxx")
        self.root.mainloop()


    def close_window(self):
        self.root.destroy()

    def update(self):
        self.draw_default(self.canva)
    
    def update_state(self,state):
        self.state = state

    def draw_default(self,canva):
        for i in range (0,canva_size,self.division_size(self.size,canva_size)):
            canva.create_line(i,0,i,canva_size, fill="black",width=2)
            canva.create_line(0,i,canva_size,i, fill="black",width=2)
        #x,y=self.game.ocuppied_pos()
        matriz=self.state
        for x in range(self.size):
            for y in range(self.size):
                if matriz[x][y]==-1:
                    color=color1
                elif matriz[x][y]==1:
                    color=color2
                elif matriz[x][y]==0:
                    color='#FFDEAD'
                else:# matriz[x][y]==3:
                    color='red'
                self.draw(canva,color,x,y)
    
    def draw(self,canva,agentcolor,x,y):
        sqr_size=self.division_size(self.size,canva_size)
        canva.create_rectangle(x*sqr_size,y*sqr_size,(x+1)*sqr_size-1,(y+1)*sqr_size-1, fill=agentcolor, outline="#8B4513")    

    def division_size(self,grid,size):
        return int(size/grid)
        
    def make_clean_canvas(self, x=700, y=700):
        base=Tk()
        base.geometry(f'{x}x{y}')
        tela=Canvas(base, bg="black", height=y,width=x)
        tela.pack()
        return base,tela

    def make_clean_frame(self,x=150,y=200):
        base=Tk()
        base.geometry(f'{x}x{y}')
        frame=ttk.Frame(base,padding=10)
        frame.grid()
        return base,frame

    def human_x_human_off(self, event):
        posicao_grelha = [event.x, event.y]
        posicao_logica = [int(posicao_grelha[0] / self.division_size(self.size,canva_size)),
                          int(posicao_grelha[1] / self.division_size(self.size,canva_size))]

        if 0 <= posicao_logica[0] < self.size and 0 <= posicao_logica[1] < self.size:
            if not self.game.is_finished(self.state, self.human_player, self.click_last_nine):
                if self.click:
                    self.x = posicao_logica[0]
                    self.y = posicao_logica[1]
                    if self.state[self.x][self.y] == self.human_player:
                        print('Valid pick')
                        self.click = False
                    else:
                        self.click = True
                        print('Invalid pick')
                else:
                    self.movex = posicao_logica[0]
                    self.movey = posicao_logica[1]

                    move = (self.x, self.y, self.movex, self.movey)
                    if self.game.verify_move(move, self.state, self.human_player):
                        # Player playing
                        self.state = self.game.play_move(move, self.state, self.human_player)
                        self.click_last_nine = self.game.add_to_lastnine(self.state, self.click_last_nine)
                        self.update()
                        self.root.update_idletasks()
                        self.human_player = -self.human_player
                        self.root.title(".BLACK." if self.human_player == 1 else ".WHITE.")
                        self.update()
                        self.root.update_idletasks()

                        if self.game.is_finished(self.state, self.human_player, self.click_last_nine):
                            winner = self.game.winner(self.state, self.human_player, self.click_last_nine)
                            winner_name = 'WHITE' if winner == 1 else ('BLACK' if winner == -1 else 'DRAW')
                            print("Game Over - Winner: " + winner_name)
                            self.root.title("Game Over - Winner: " + winner_name)
                            time.sleep(5)
                            self.root.destroy()

                        self.click = True  # Switch player turns
                    else:
                        print("Invalid move")
                        self.root.title("INVALID MOVE...")
                        self.click = True

                    self.click = True
                    self.root.update()
            else:
                winner = self.game.winner(self.state, self.human_player, self.click_last_nine)
                winner_name = 'WHITE' if winner == 1 else ('BLACK' if winner == -1 else 'DRAW')
                print("Game Over - Winner: " + winner_name)
                self.root.title("Game Over - Winner: " + winner_name)
                time.sleep(5)
                self.root.destroy()
        else:
            print("Pick or move out of range")
            self.root.title("INVALID POSITION...")

    def pc_x_pc_off_random(self,s):
        player = 1
        last_nine = [s]
        print('start_game')
        print(s)

        def play_next_move(s, last_nine , player):
            if not self.game.is_finished(s, player, last_nine):

                print('white' if player == 1 else 'black')
                move = self.game.get_random_move(s,player)
                s = self.game.play_move(move, s, player)
                last_nine = self.game.add_to_lastnine(s, last_nine)
                print('move: ', move)
                print(s)

                self.update_state(s)
                self.update()
                player = -player
                #this works as a cilce with time.slepp of 1000 miliseconds
                self.root.after(self.time_between_moves, lambda: play_next_move(s, last_nine, player))   
            else:
                winner = self.game.winner(s, player, last_nine)
                winner_name = 'white' if winner == 1 else ('black' if winner == -1 else 'draw')
                print("Game Over - Winner: " + winner_name)
                self.root.title("Game Over - Winner: " + winner_name)

        self.update_state(s)
        self.update()
        self.root.after(self.time_between_moves, lambda: play_next_move(s, last_nine, player))
        self.root.mainloop()

    def human_vs_pc_off_random(self, event):
        posicao_grelha = [event.x, event.y]
        posicao_logica = [int(posicao_grelha[0]/self.division_size(self.size,canva_size)),int(posicao_grelha[1]/self.division_size(self.size,canva_size))]

        if 0 <= posicao_logica[0] < self.size and 0 <= posicao_logica[1] < self.size:
            player = 1 #we start with the white
            if not self.game.is_finished(self.state, player, self.click_last_nine):

                if self.click==True:
                    self.x=posicao_logica[0]
                    self.y=posicao_logica[1]
                    if self.state[self.x][self.y]==player:
                        print('valid pick')
                        self.click=False
                    else:
                        self.click=True
                        print('invalid pick')
                else:
                    self.movex=posicao_logica[0]
                    self.movey=posicao_logica[1]

                    move = (self.x,self.y,self.movex,self.movey)

                    #after getting all info to play
                    if self.game.verify_move(move,self.state,player):
                        #me playing
                        self.state = self.game.play_move(move, self.state, player)
                        self.click_last_nine = self.game.add_to_lastnine(self.state, self.click_last_nine)
                        self.update()
                        self.root.update_idletasks()
                        player = -player
                        self.root.title(".BLACK.")
                        self.update()
                        self.root.update_idletasks()

                        if self.game.is_finished(self.state, player, self.click_last_nine):
                            winner = self.game.winner(self.state, player, self.click_last_nine)
                            winner_name = 'white' if winner == 1 else ('black' if winner == -1 else 'draw')
                            self.update()
                            self.root.update_idletasks()
                            print("Game Over - Winner: " + winner_name)
                            self.root.title("Game Over - Winner: " + winner_name)
                            time.sleep(5)
                            self.root.destroy()

                        #pc playing
                        time.sleep((self.time_between_moves/1000)) #wait 1 second
                        move = self.game.get_random_move(self.state,player)
                        self.state = self.game.play_move(move, self.state, player)
                        self.click_last_nine = self.game.add_to_lastnine(self.state, self.click_last_nine)
                        self.root.title("Ataxx: White playing")
                        self.update()
                        self.root.update_idletasks()
                        player = -player
                        self.click=True 

                    else:
                        print("invalid move")
                        self.root.title("INVALID MOVE...")
                        self.click=True

                    self.click=True
                    self.root.update()
            else:
                winner = self.game.winner(self.state, player, self.click_last_nine)
                winner_name = 'white' if winner == 1 else ('black' if winner == -1 else 'draw')
                self.update()
                self.root.update_idletasks()
                print("Game Over - Winner: " + winner_name)
                self.root.title("Game Over - Winner: " + winner_name)
                self.update()
                self.root.update_idletasks()
                time.sleep(5)
                self.root.destroy()
        else:
            print("pick or move out of range")
            self.root.title("INVALID POSITION...")

    def pc_x_pc_on_random(self,s):
        last_nine = [s]
        player = 1
        print('start_game')
        print(s)

        def play_next_move(s, last_nine, player):
            if not self.game.is_finished(s,player,last_nine):   

                if self.ag == 1 or not self.first:
            # 1_PLAY & SEND___________________________________
                    
                    self.root.title(":self playing: " + self.game.get_color(player))
                    print('white' if player == 1 else 'black')
                    move = self.game.get_random_move(s,player)
                    s = self.game.play_move(move, s, player)
                    last_nine = self.game.add_to_lastnine(s, last_nine)
                    print('move: ', move)
                    print(s)
                    player = -player

                    #interface updates
                    self.root.title(":opponent playing: " + self.game.get_color(player))
                    self.update_state(s)
                    self.update()
                    self.root.update_idletasks()
                    self.root.title(":opponent playing: " + self.game.get_color(player))
        
                    #SEND MOVE
                    move_message = self.client.stringify_move_ataxx(move[0], move[1], move[2], move[3])
                    time.sleep(1)
                    self.client.client_socket.send(move_message.encode())
                    print("Send:", move_message)


            # 2_RECEIVE & PLAY___________________________________
                    # play as the other side
                    response = self.client.client_socket.recv(1024).decode()
                    print(f"Server Response1: {response}")
                    if "END" in response:
                        return #else:

                    print('white' if player == 1 else 'black')
                    move = self.client.decode_stringify_move_ataxx(response)
                    s = self.game.play_move(move, s, player)
                    last_nine = self.game.add_to_lastnine(s, last_nine)
                    print('move: ', move)
                    print(s)
                    player = -player

                    #interface updates
                    self.root.title(":self playing: " + self.game.get_color(player))
                    self.update_state(s)
                    self.update()
                    self.root.update_idletasks()
                    self.root.title(":self playing: " + self.game.get_color(player))

    
                else: # the other client
                    self.first = False
                    self.root.title(":opponent playing: " + self.game.get_color(player))
                    #the other don't change the player, because he will play for our side first.
                    response = self.client.client_socket.recv(1024).decode()
                    print(f"Server Response2: {response}")
    
                    if "END" in response:
                        return
    
                    print('white' if player == 1 else 'black')
                    move = self.client.decode_stringify_move_ataxx(response)
                    s = self.game.play_move(move, s, player)
                    last_nine = self.game.add_to_lastnine(s, last_nine)
                    print('move: ', move)
                    print(s)
                    player = -player

                    #interface updates
                    self.root.title(":self playing: " + self.game.get_color(player))
                    self.update_state(s)
                    self.update()
                    self.root.update_idletasks()
                    self.root.title(":self playing: " + self.game.get_color(player))
                    

                # Schedule the next move after a delay (in milliseconds)
                time.sleep(1)
                self.root.after(self.time_between_moves, lambda: play_next_move(s, last_nine, player))

            else:
                #the agent 1 is always the white
                winner = self.game.winner(s, player, last_nine)
                winner_name = 'white, Ag1' if winner == 1 else ('black, Ag2' if winner == -1 else 'draw')
                print("Game Over - Winner: " + winner_name)
                self.root.title("Game Over - Winner: " + winner_name)
                self.client.client_socket.close()
                return
            
        self.update_state(s)
        self.update()
        if self.ag == 1:
            self.root.title(":self playing: " + self.game.get_color(player))
        else:
            self.root.title(":opponent playing: " + self.game.get_color(player))
        self.root.after(self.time_between_moves, lambda: play_next_move(s, last_nine, player))
        self.root.mainloop()

# Go

In [6]:
class Go:

    def __init__(self,size):
        self.size = size
        self.komi = 5.5             
        self.pass_move = (-1,-1)    #this out of range move is used to pass the turn
        #action size: total board + pass move
        self.action_size = self.size * self.size + 1 
    
    def make_default_s0(self):
        #used only for testing and debugging
        return np.zeros((self.size, self.size), dtype=int)
    
    def place(self,x,y,state,player):
        state = state.copy()                                    #copy()
        state[x][y] = player
        return state
    
    def remove(self,x,y,state):
        state = state.copy()                                    #copy()
        state[x][y] = 0
        return state
    
    def get_color(self,player):
        return 'black' if player == 1 else 'white'
    
    def get_color_of_position(self,x,y,state):
        return self.get_color(state[x][y])
    
    def isfree(self,x,y,state):                               
        return state[x][y]==0
    
    def get_neighbors(self, x, y) -> [(int, int)]:
        #returns all (max: 4) neigbours, empty or not, inside board range 
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        liberties = [(x + dx, y + dy) for dx, dy in directions if 0 <= x + dx < self.size and 0 <= y + dy < self.size]
        return liberties

    def get_liberties(self,x,y,state):
        #returns empty neigbours
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        liberties = [(x + dx, y + dy) for dx, dy in directions if (0 <= x + dx < self.size and 0 <= y + dy < self.size and state[x + dx][y + dy] == 0)]
        return liberties
    
    def has_liberties(self,x,y,state):
        return  len(self.get_liberties(x,y,state)) > 0
    
    def group_has_liberty(self,group,state):
        #if group is not limited
        for stone in group:
            x, y = stone
            liberties = self.get_liberties(x, y, state)
            if liberties:
                    return True
        return False
    
    def identify_limited_groups_by_player(self,state, player) -> [{(int, int)}]: 
        #retorna apenas grupos sem liberties
        #using dfs, expande until no more of the same color
        state = state.copy()
    #{
        limited_group_list = []
        value = player
        positions = list(map(tuple, np.argwhere(state == value)))
        visited = set()
        group = set()        
        def identify_group(x,y,state):
        #{  um grupo que cotem x,y
            def dfs(a,b,state):
            #{  propagação por dfs
                sx, sy = a, b
                if (
                    0 <= sx < self.size
                    and 0 <= sy < self.size
                    and (sx, sy) not in visited
                    and state[sx][sy] == state[x][y]
                ):
                    visited.add((sx, sy))
                    group.add((sx, sy))
                    for neighbour in self.get_neighbors(sx, sy):
                        dfs(neighbour[0], neighbour[1],state)
            #{
            if 0 <= x < self.size and 0 <= y < self.size:
                dfs(x, y,state)
        #}
        for (x,y) in positions:
            if (x,y) not in visited:
                identify_group(x,y,state)
                if not self.group_has_liberty(group,state): #Está verificado se grupo está preso
                    limited_group_list.append(group)    
                group = set() #reset group
    #}
        return limited_group_list
    
    def identify_group_by_position(self, x, y, state):
        #to avoid checking all board in some plays
        visited = set()
        group = set()
        def dfs(a, b, state):
            # o dfs está aqui dentro porque precisa do x,y
            sx, sy = a, b
            if (
                0 <= sx < self.size
                and 0 <= sy < self.size
                and (sx, sy) not in visited
                and state[sx][sy] == state[x][y]
            ):
                visited.add((sx, sy))
                group.add((sx, sy))
                for neighbour in self.get_neighbors(sx, sy):
                    dfs(neighbour[0], neighbour[1])
        if 0 <= x < self.size and 0 <= y < self.size:
            dfs(x, y, state)
        return group

    def capture_all(self, limited_group_list, state):
        #if limited group after a play,it might be captured, 
        #temos que retornar peças capturadas, para subtrair no final
        captures = {1: 0, -1: 0}
        state = state.copy()                                                            
        for group in limited_group_list:
            for (x,y) in group:
                if state[x][y] == 1:
                    captures[1] += 1
                if state[x][y] == -1:
                    captures[-1] += 1
                state[x][y] = 0

        return state, captures
        
    def add_to_last_three(self, state, last_three):
        #we are doing a lot of copies, because of mtcs branching
        last_three = last_three.copy()                           #copy()
        state = state.copy()
        if (len(last_three) >= 3):
            last_three.pop(0)
            last_three.append(state)
        else:
            last_three.append(state)
        return last_three
        
    def is_Ko(self, next_state, last_three): 
        #verify ko rule by comparing to the penultimate state
        last_three = last_three.copy()
        # !!-> state corresponde ao próximo estado <-!! #
        if len(last_three) == 3:
            b = np.array_equal(last_three[1],next_state)
            return b
        elif len(last_three) == 2:
            b = np.array_equal(last_three[0],next_state)
            return b
        else:
            return False
    
    def is_valid_move(self, move, state, player, last_three):
        #check Suicide and Ko rules
        last_three = last_three.copy()
        state = state.copy()
        moveX, moveY = move[0] , move[1]

        if (moveX,moveY) == (-1,-1):
            return True
        #se posição é vazia
        if state[moveX][moveY] == 0:
            
            #First we place
            middle_state = self.place(moveX, moveY, state, player) 

            #verify if our move would captures opponent
            opponent_verify = 1 if player == -1 else -1
            opponent_limited_groups = self.identify_limited_groups_by_player(middle_state, opponent_verify)
            opponent_capture = True if opponent_limited_groups else False  

            #verify if our move would captures ourself(SUICIDE verification)
            self_limited_groups = self.identify_limited_groups_by_player(middle_state, player)
            self_capture = True if self_limited_groups else False
            #if only capture ourself -> suicide
            isSuicide = (not opponent_capture and self_capture)  
            if not isSuicide:
                future_resulting_not_suicide_state, _  = self.capture_all(opponent_limited_groups,middle_state)
                isKo = self.is_Ko(future_resulting_not_suicide_state, last_three)
            else:
                return False
            
            is_valid = ((not isSuicide) and (not isKo))
            return is_valid
        else: 
            return False
            
        
    def action_space(self,state, player, last_three): #---------------------------->DEAL WITH PASS MOVE
        #veryfing Ko and suicide and empty places
        action_space = []
        possible_moves = list(map(tuple, np.argwhere(state == 0)))
        for move in possible_moves:
            isValid = self.is_valid_move(move, state, player, last_three)
            if isValid:
                action_space.append(move)
        action_space.append(self.pass_move)
        return action_space
    
    def play_move(self,move,state,player): 
        state = state.copy()
        #**ELE_JÁ_VEM_VERIFICADO_REMOVER_EM_PRINCPIO**#
        if move == (-1,-1) or move is None:
            return state, {1:0,-1:0}
        else:
            x = move[0]
            y = move[1]
            state = self.place(x,y,state,player)
            capture = -1 if player == 1 else 1
            limited_group_list = self.identify_limited_groups_by_player(state, capture)
            state, captures = self.capture_all(limited_group_list,state)
            return state, captures

    def get_random_move(self, state, player, last_three):
        action_space = self.action_space(state, player, last_three)
        if action_space:
            move = action_space[np.random.randint(0,len(action_space))]
            return move
        else:
        #Fazer algo apropriado se action_space vazio
             print("A lista action_space está vazia.")
             
    def is_finished(self, state, last_three):
        state = state.copy()
        #if full board or both passes -> (last_three staet are equal)
        if len(last_three) < 3:
            return False
        first = last_three[0]
        second = last_three[1]
        third = last_three[2]
        c, _ = np.where(state == 0)
        return len(c) == 0 or (np.array_equal(first, second) and np.array_equal(second, third))

    def identify_territory_by_player(self, state, player) -> [{(int, int)}]:
    #we start by identifying all territories based on his probable owner. By player
        state = state.copy()
        #liberties visitadas + peças visitadas
        liberties_visited = set()
        player_pieces_visited = set()
        #territorio (empty positions) + peças do jogador
        territories = [] # list of sets of tuples os 2 ints
        player_pieces = list(map(tuple, np.argwhere(state == player)))

        #identify empty groups by liberty position
        def identify_territory_by_liberty_position(x, y, state):
            state = state.copy()
            territory = set()
            #dfs search
            def dfs(a, b, state):
                sx, sy = a, b
                if (
                    0 <= sx < self.size
                    and 0 <= sy < self.size
                    and (sx, sy) not in liberties_visited
                    and (state[sx][sy] == 0)
                ):
                    liberties_visited.add((sx, sy))
                    territory.add((sx, sy))
                    #apenas se tivermos liberty disponivel é que continuamos a expansão
                    for libs in self.get_liberties(sx, sy, state):  
                        dfs(libs[0], libs[1], state)
            #aqui chamamos dfs          
            if 0 <= x < self.size and 0 <= y < self.size:
                dfs(x, y, state)
            return territory

        #start from empty positions near player pieces
        for (x,y) in player_pieces:
            if (x,y) not in player_pieces_visited:
                player_pieces_visited.add((x,y))
                #get liberties of player pieces and start search with one of them
                liberties = self.get_liberties(x,y,state)
                for (x,y) in liberties:
                    if (x,y) not in liberties_visited:
                        territory = identify_territory_by_liberty_position(x,y,state)
                        territories.append(territory)
                    
        return territories

    def check_territory_owner(self, territory, state, player):
    #each territory:a) identify the owner of a territory, b)we highlight(9,-9,5) the positions that belong to the owner & return it. c)count and remove the dead pieces on enemy territory. 
        state = state.copy()
        null_state = state.copy()
        my_state = state.copy()
        opponent_state = state.copy()
        me = player
        opponent = -1 if player == 1 else 1

        #fill territory with opponent pieces, and observe...
        for (x, y) in territory:
            state[x][y] = opponent

        #Listas de grupos limitados. Por jogador
        opponent_limited_created = self.identify_limited_groups_by_player(state, opponent)
        me_limited_created = self.identify_limited_groups_by_player(state, me)

        #Se com o fill, nenhum limited criado -> territorio neutro, e preenchemos com numero 0
        if len(opponent_limited_created) == 0 and len(me_limited_created) == 0:
            for (x,y) in territory:
                null_state[x][y] = 5
            return {'owner': 0,'masked': null_state,'removed': 0} #Dono, masked with neutral: 0, pieces removed.
        
        #se só adeversário perde com o fill, então territorio é meu
        elif len(opponent_limited_created) >0 and len(me_limited_created) == 0:
            opponent_removed_pieces = -len(territory) #a contar com peças que já estavam lá
            for each_fill in opponent_limited_created:
                for (x,y) in each_fill:
                    opponent_removed_pieces+=1
                    my_state[x][y] = me*9   #estes territórios são meus
            return {'owner': me,'masked': my_state,'removed': opponent_removed_pieces}
        
        #se só eu perco com o fill, então territorio é do adeversário, e as minhas peças são removidas e subtraidas no meu território total
        elif len(opponent_limited_created) == 0 and len(me_limited_created) > 0:
            my_removed_pieces = -len(territory)
            for each_fill in me_limited_created:
                for (x,y) in each_fill:
                    my_removed_pieces+=1
                    opponent_state[x][y] = opponent*9
            return {'owner': opponent,'masked': opponent_state,'removed': my_removed_pieces}
        
        else:
            #Para casos Dubios.
            #Ambos > 0 -> para verificar esta condição, vamos fazer um novo fill, com as minhas peças:
            #se eu for capturado, é do adeversário, se eu não for, é meu 
            for (x, y) in territory:
                state[x][y] = me
            opponent_limited_created = self.identify_limited_groups_by_player(state, opponent)
            me_limited_created = self.identify_limited_groups_by_player(state, me)
            #np.where -> ([],[])

            #se eu for capturado, é do adeversário
            if len(me_limited_created) >0 and len(opponent_limited_created) == 0:
                #se eu não for capturado, é meu
                my_removed_pieces = -len(territory)
                for each_fill in me_limited_created:
                    for (x,y) in each_fill:
                        my_removed_pieces+=1
                        opponent_state[x][y] = opponent*9
                return {'owner': opponent,'masked': opponent_state,'removed': my_removed_pieces}

            #se eu não for capturado, é meu
            else:
                for (x,y) in territory:
                    my_state[x][y] = me*9
                return {'owner': me,'masked': my_state,'removed': 0}
            
    def winner(self, state, during_game_captures):
    #receives state & during game captures dict('player': n_captures)
        black_captures = during_game_captures[1]
        white_captures = during_game_captures[-1]
        state = state.copy()
        final_state = state.copy()

        #we start by defining the possibe ownner of all territories
        white_territories = self.identify_territory_by_player(state, -1)
        black_territories = self.identify_territory_by_player(state, 1)
        both = [white_territories,black_territories]

        #then we verify it
        neutral_masks = []
        white_masks = []
        black_masks = []

        #get_information
        for possible_own in both:
            for territory in possible_own:
                territory_info = self.check_territory_owner(territory,state,-1)

                owner = territory_info['owner']
                s = territory_info['masked']
                removed = territory_info['removed']

                if owner == 0:
                    neutral_masks.append(s.copy())
                elif owner == -1:
                    white_masks.append(s.copy())
                    white_captures += removed
                else:
                    black_masks.append(s.copy())
                    black_captures += removed
        
        #apply masks -> 2º white, 3º black
        for mask in neutral_masks:
            #Será que 5 não prejudica o funcionamento de outras funções?????
            neutral_points = list(map(tuple, np.argwhere(mask == 5)))
            for (x,y) in neutral_points:
                final_state[x][y] = 5
        
        for mask in white_masks:
            white_points = list(map(tuple, np.argwhere(mask == -9)))
            for (x,y) in white_points:
                final_state[x][y] = -9

        for mask in black_masks:
            black_points = list(map(tuple, np.argwhere(mask == 9)))
            for (x,y) in black_points:
                final_state[x][y] = 9

        #count points
        white_points = self.komi + len(np.where(final_state == -9)[0]) - white_captures
        black_points = len(np.where(final_state == 9)[0]) - black_captures
        winner = 1 if black_points > white_points else (-1 if white_points > black_points else 0)
        #np.where -> ([],[])

        print('____final_counting___')
        print()
        print('Black captured:', black_captures, end=' ')
        print('White captured:', white_captures)
        print()
        print('Black points:', black_points, end=' ')
        print('White points:', white_points)
        print()
        print('________Winner_______: ', self.get_color(winner), ' by', abs(black_points - white_points), ' difference')
        print(final_state)

        return winner

# interface Go

In [7]:
class interfaceGo():

    def __init__(self, size, game, initial_state, type_game, time_between_moves, client = None, ag = None):
        self.size = size
        self.game = game
        self.state = initial_state
        self.time_between_moves = time_between_moves
        #playing with comunication
        if client:
            self.comunication = True
            self.client = client
            self.ag=ag      #se é agente 1 ou 2
            self.first=True
            type_game = 3

        #game features that need to be external to the game
        self.human_last_three = []
        self.human_captures = {1: 0, -1: 0}
        self.human_player = 1

        self.root, self.canva = self.make_clean_canvas(canva_size)
        self.draw_default(self.canva)
        
        if(type_game==1):   
            #human human
            self.root.bind("<Button-1>", self.human_x_human_off)
            self.pass_button = ttk.Button(self.root, text="Pass", command=self.make_pass_move_human_x_human)
            #pass button
            self.pass_button.pack(side=BOTTOM, pady=10)
        elif(type_game==2):
            #pc human
            self.root.bind("<Button-1>", self.human_x_pc_off_random)
            self.pass_button = ttk.Button(self.root, text="Pass", command=self.make_pass_move_human_x_pc)
            self.pass_button.pack(side=BOTTOM, pady=10)
        elif(type_game==3):
            #pc pc with or without comunication
            if client: 
                self.pc_x_pc_on_random(initial_state)
            else: #ofline
                self.pc_x_pc_off_random(initial_state)

        self.root.title("Go")
        self.root.mainloop()

    def close_window(self):
        self.root.destroy()

    def update(self):
        self.canva.destroy()                
        self.canva=Canvas(self.root, bg="#FFDEAD", height=canva_size,width=canva_size)
        self.canva.pack()
        self.draw_default(self.canva)
    
    def update_state(self,state):
        self.state = state

    def draw_default(self,canva):
        for i in range (0,canva_size,self.division_size(self.size,canva_size)):
            canva.create_line(i+go_offset,go_offset,i+go_offset,canva_size+go_offset-self.division_size(self.size,canva_size), fill="black",width=4)
            canva.create_line(go_offset,i+go_offset,canva_size+go_offset-self.division_size(self.size,canva_size),i+go_offset, fill="black",width=4)
            #time.sleep(5)
        #x,y=self.mk.ocuppied_pos()
        matriz=self.state
        for x in range(self.size):
            for y in range(self.size):
                if matriz[x][y]==1:
                    color=color1 #preto
                    self.draw(canva,color,x,y)
                elif matriz[x][y]==-1:  #branco
                    color=color2
                    self.draw(canva,color,x,y)
                else:#elif matriz[x][y]==0:
                    pass
    
    def draw(self,canva,agentcolor,x,y):
        sqr_size=self.division_size(self.size,canva_size)
        canva.create_oval((x*sqr_size)-(sqr_size/2.5)+go_offset,(y*sqr_size)-(sqr_size/2.5)+go_offset,((x+1)*sqr_size)-(sqr_size/1.5)+go_offset,((y+1)*sqr_size)-(sqr_size/1.5)+go_offset, fill=agentcolor)        

    def division_size(self,grid,size):
        return round(size/grid)
        
    def make_clean_canvas(self, x=canva_size, y=canva_size):
        base=Tk()
        base.geometry(f'{x+100}x{y+100}')
        #tela=Canvas(base, bg="white", height=y-self.division_size(grid_size,canva_size),width=x-self.division_size(grid_size,canva_size))
        tela=Canvas(base, bg="#FFDEAD", height=y,width=x)
        tela.pack()
        return base,tela

    def make_clean_frame(self,x=150,y=200):
        base=Tk()
        base.geometry(f'{x}x{y}')
        frame=ttk.Frame(base,padding=10)
        frame.grid()
        return base,frame

    def make_pass_move_human_x_human(self):
        #if you click pass, this is activated, move =(-1,-1)
        print('PASS')
        state = self.state.copy()
        player = self.human_player
        #if not finished
        if not self.game.is_finished(state, self.human_last_three):
            #jogamos e recebemos resulting state and captures. UPDATE IT
            #UPDATE captures and last_three
            self.human_last_three = self.game.add_to_last_three(state, self.human_last_three)
            #update interface and players
            self.update()
            self.root.update_idletasks()
            self.human_player = -player
            self.root.title(".BLACK." if self.human_player == 1 else ".WHITE.")
            self.update()
            self.root.update_idletasks()
        else:
            winner = self.game.winner(self.state, self.human_captures)
            winner_name = 'WHITE' if winner == -1 else ('BLACK' if winner == 1 else 'DRAW')
            print("Game Over - Winner: " + winner_name)
            self.root.title("Game Over - Winner: " + winner_name)
            return

    def human_x_human_off(self, event):
        posicao_grelha = [event.x - go_offset+20, event.y - go_offset+20]
        posicao_logica = [int(posicao_grelha[0] / self.division_size(self.size, canva_size)),
                          int(posicao_grelha[1] / self.division_size(self.size, canva_size))]
        
          # Define os limites do tabuleiro de jogo
        board_left = go_offset - 20
        board_right = go_offset + 20 + self.size * self.division_size(self.size, canva_size)
        board_top = go_offset - 20
        board_bottom = go_offset + 20 + self.size * self.division_size(self.size, canva_size)
         # Verifica se o evento de clique ocorreu dentro do tabuleiro de jogo
        if not (board_left <= event.x <= board_right and board_top <= event.y <= board_bottom):
            print("cursor out of range")
            return

        if 0 <= posicao_logica[0] < self.size and 0 <= posicao_logica[1] < self.size:
            #save move
            move = (posicao_logica[0], posicao_logica[1])
            #verify if game is not finished
            if not self.game.is_finished(self.state, self.human_last_three):
                #verify if move is valid
                if self.game.is_valid_move(move, self.state, self.human_player, self.human_last_three.copy()):
                    #play and update
                    self.state, c = self.game.play_move(move, self.state, self.human_player)
                    self.human_last_three = self.game.add_to_last_three(self.state, self.human_last_three)
                    self.human_captures[1] += c[1]
                    self.human_captures[-1] += c[-1]
                    self.update()
                    self.root.update_idletasks()
                    self.human_player = -self.human_player
                    self.root.title(".BLACK." if self.human_player == 1 else ".WHITE.")
                    self.update()
                    self.root.update_idletasks()
                  
            else:
                winner = self.game.winner(self.state, self.human_captures)
                winner_name = 'WHITE' if winner == -1 else ('BLACK' if winner == 1 else 'DRAW')
                print("Game Over - Winner: " + winner_name)
                self.root.title("Game Over - Winner: " + winner_name)
                return
        else:
            print('logic position out of range')


    def make_pass_move_human_x_pc(self):
        player = 1 #we start with the black
        move = (-1,-1)
        print('PASS')


        #if not finished
        if not self.game.is_finished(self.state, self.human_last_three):
            #jogamos e recebemos resulting state and captures. UPDATE IT
            #UPDATE captures and last_three
            self.human_last_three = self.game.add_to_last_three(self.state, self.human_last_three)

            #update interface and players
            self.update()
            self.root.update_idletasks()
            #passar a jogada ao pc
            player = -player
            self.root.title(".BLACK." if self.human_player == 1 else ".WHITE.")
            self.update()
            self.root.update_idletasks()

        else:
            #in case of end game
            winner = self.game.winner(self.state, self.human_captures)
            winner_name = 'WHITE' if winner == -1 else ('BLACK' if winner == 1 else 'DRAW')
            print("Game Over - Winner: " + winner_name)
            self.root.title("Game Over - Winner: " + winner_name)
            return

        if not self.game.is_finished(self.state, self.human_last_three):
            #PC playing
            time.sleep((self.time_between_moves/1000)) #wait x seconds
            #we ask for a move, so it's forcelly a valid move
            move = self.game.get_random_move(self.state,player,self.human_last_three)
            #update state and captures
            self.state, c = self.game.play_move(move, self.state, player)
            self.human_last_three = self.game.add_to_last_three(self.state,self.human_last_three)
            self.human_captures[1] += c[1]
            self.human_captures[-1] += c[-1]

            #interface
            self.update()
            self.root.update_idletasks()
            self.root.title(".BLACK." if self.human_player == 1 else ".WHITE.")
            self.update()
            self.root.update_idletasks()
        else:
            #in case of end game
            winner = self.game.winner(self.state, self.human_captures)
            winner_name = 'white' if winner == 1 else ('black' if winner == -1 else 'draw')
            self.update()
            self.root.update_idletasks()
            print("Game Over - Winner: " + winner_name)
            self.root.title("Game Over - Winner: " + winner_name)
            return

    def human_x_pc_off_random(self, event):
        posicao_grelha = [event.x, event.y]
        posicao_logica = [int(posicao_grelha[0]/self.division_size(self.size,canva_size)),int(posicao_grelha[1]/self.division_size(self.size,canva_size))]

        #Define board limits
        board_left = go_offset - 20
        board_right = go_offset + 20 + self.size * self.division_size(self.size, canva_size)
        board_top = go_offset - 20
        board_bottom = go_offset + 20 + self.size * self.division_size(self.size, canva_size)
          # Verifica se o evento de clique ocorreu dentro do tabuleiro de jogo
        if not (board_left <= event.x <= board_right and board_top <= event.y <= board_bottom):
            print("cursor out of range")
            return

        if 0 <= posicao_logica[0] < self.size and 0 <= posicao_logica[1] < self.size:
            player = 1 #we start with the black
            move = (posicao_logica[0], posicao_logica[1])
            #verify if game is not finished
            if not self.game.is_finished(self.state,self.human_last_three):

                if self.game.is_valid_move(move, self.state, self.human_player, self.human_last_three):
                        
                        self.state, c = self.game.play_move(move, self.state, player)
                        self.human_last_three = self.game.add_to_last_three(self.state.copy(),self.human_last_three)
                        self.human_captures[1] += c[1]
                        self.human_captures[-1] += c[-1]
                        #update window after and before change player
                        self.update()
                        self.root.update_idletasks()
                        player = -player
                        self.root.title(".BLACK." if self.human_player == 1 else ".WHITE.")
                        self.update()
                        self.root.update_idletasks()
            
                        #pc playing
                        time.sleep((self.time_between_moves/1000)) #wait x seconds
                        move = self.game.get_random_move(self.state,player,self.human_last_three)
                        if move == (-1,-1):
                            print('pc PASS')
                        self.state, c = self.game.play_move(move, self.state, player)
                        self.human_last_three = self.game.add_to_last_three(self.state.copy(),self.human_last_three)
                        self.human_captures[1] += c[1]
                        self.human_captures[-1] += c[-1]
                        self.update()
                        self.root.update_idletasks()
                        player = -player
                        self.root.title(".BLACK." if self.human_player == 1 else ".WHITE.")
        
                else:
                    print("invalid move")
                    self.root.title("INVALID MOVE...")
                    self.root.update()
            else:
                winner = self.game.winner(self.state, self.human_captures)
                winner_name = 'white' if winner == 1 else ('black' if winner == -1 else 'draw')
                self.update()
                self.root.update_idletasks()
                print("Game Over - Winner: " + winner_name)
                self.root.title("Game Over - Winner: " + winner_name)
                return
        else:
            print("pick or move out of range")


    def pc_x_pc_off_random(self,s):
        player = 1
        last_three = [s]
        captures = {1: 0, -1: 0}
        print('start_game')
        print(s)

        def play_next_move(s, player, last_three, captures):
            if not self.game.is_finished(s, last_three):

                print('white' if player == -1 else 'black')
                move = self.game.get_random_move(s,player, last_three)
                if move == (-1,-1):
                    print('Pass')
                    print('move: ', move)
                    print(s)
                    last_three = self.game.add_to_last_three(s, last_three)
                    player = -player
                    self.update_state(s)
                    self.update()
                    self.root.after(self.time_between_moves, lambda: play_next_move(s, player, last_three, captures))
                else:
                    s, c = self.game.play_move(move, s, player)
                    last_three = self.game.add_to_last_three(s, last_three)
                    captures[1] += c[1]
                    captures[-1] += c[-1]
                    print('move: ', move)
                    print(s)

                    self.update_state(s)
                    self.update()
                    player = -player
                     #this works as a cilce with time.slepp of 1000 miliseconds
                    self.root.after(self.time_between_moves, lambda: play_next_move(s, player, last_three, captures))
            else:
                winner = self.game.winner(s, captures)
                winner_name = 'white' if winner == -1 else ('black' if winner == 1 else 'draw')
                print("Game Over - Winner: " + winner_name)
                self.root.title("Game Over - Winner: " + winner_name)

        self.update_state(s)
        self.update()
        self.root.after(self.time_between_moves, lambda: play_next_move(s, player, last_three, captures))
        self.root.mainloop()

    def pc_x_pc_on_random(self,s):
        player = 1
        last_three = [s]
        captures = {1: 0, -1: 0}
        print('start_game')
        print(s)

        def play_next_move(s, last_three, player, captures):
            if not self.game.is_finished(s, last_three):   

                if self.ag == 1 or not self.first:
            # 1_PLAY & SEND___________________________________
                    
                    self.root.title(":self playing: " + self.game.get_color(player))
                    print('white' if player == -1 else 'black')
                    move = self.game.get_random_move(s, player, last_three)
                    s, c = self.game.play_move(move, s, player)
                    last_three = self.game.add_to_last_three(s, last_three)
                    captures[1] += c[1]
                    captures[-1] += c[-1]
                    print('move: ', move)
                    print(s)
                    player = -player

                    #interface updates
                    self.root.title(":opponent playing: " + self.game.get_color(player))
                    self.update_state(s)
                    self.update()
                    self.root.update_idletasks()
                    self.root.title(":opponent playing: " + self.game.get_color(player))
        
                    #SEND MOVE
                    move_message = self.client.stringify_move_go(move[0], move[1])
                    time.sleep(1)
                    self.client.client_socket.send(move_message.encode())
                    print("Send:", move_message)


            # 2_RECEIVE & PLAY___________________________________
                    # play as the other side
                    if not self.game.is_finished(s, last_three):   
                        response = self.client.client_socket.recv(1024).decode()
                        print(f"Server Response1: {response}")
                        if "END" in response:
                            return #else:

                        print('white' if player == -1 else 'black')
                        move = self.client.decode_stringify_move_go(response)
                        s, c = self.game.play_move(move, s, player)
                        last_three = self.game.add_to_last_three(s, last_three)
                        captures[1] += c[1]
                        captures[-1] += c[-1]

                        print('move: ', move)
                        print(s)
                        player = -player

                        #interface updates
                        self.root.title(":self playing: " + self.game.get_color(player))
                        self.update_state(s)
                        self.update()
                        self.root.update_idletasks()
                        self.root.title(":self playing: " + self.game.get_color(player))
                    else:
                        winner = self.game.winner(s, captures)
                        winner_name = 'black, Ag1' if winner == 1 else ('white, Ag2' if winner == -1 else 'draw')
                        print("Game Over - Winner: " + winner_name)
                        self.root.title("Game Over - Winner: " + winner_name)
                        self.client.client_socket.close()
                        return

    
                else: # the other client
                    print('other')
                    self.first = False
                    self.root.title(":opponent playing: " + self.game.get_color(player))
                    #the other don't change the player, because he will play for our side first.
                    response = self.client.client_socket.recv(1024).decode()
                    print(f"Server Response2: {response}")
    
                    if "END" in response:
                        return
    
                    print('white' if player == -1 else 'black')
                    move = self.client.decode_stringify_move_go(response)
                    s, c = self.game.play_move(move, s, player)
                    last_three = self.game.add_to_last_three(s, last_three)
                    captures[1] += c[1]
                    captures[-1] += c[-1]
                    print('move: ', move)
                    print(s)
                    player = -player

                    #interface updates
                    self.root.title(":self playing: " + self.game.get_color(player))
                    self.update_state(s)
                    self.update()
                    self.root.update_idletasks()
                    self.root.title(":self playing: " + self.game.get_color(player))
                    

                # Schedule the next move after a delay (in milliseconds)
                time.sleep(1)
                self.root.after(self.time_between_moves, lambda: play_next_move(s, last_three, player, captures))

            else:
                #the agent 1 is always the white
                winner = self.game.winner(s, captures)
                winner_name = 'black, Ag1' if winner == 1 else ('white, Ag2' if winner == -1 else 'draw')
                print("Game Over - Winner: " + winner_name)
                self.root.title("Game Over - Winner: " + winner_name)
                self.client.client_socket.close()
                return
            
        self.update_state(s)
        self.update()
        if self.ag == 1:
            self.root.title(":self playing: " + self.game.get_color(player))
        else:
            self.root.title(":opponent playing: " + self.game.get_color(player))
        self.root.after(self.time_between_moves, lambda: play_next_move(s, last_three, player, captures))
        self.root.mainloop()

# Socket-based communication

class Client suports the client connection and is responsible for defining encode and decode functions as well as receiving and treating information sent by the server

To play with communication:
>first: edit configurations 
>
>start the server
>
>then call 2 seperatly mains()

##### client

In [8]:
# client
class Client:
    def __init__(self, host=None, port = None):  # Defina PORT conforme necessário
        self.host = host
        self.port = port
        self.ag=1

    #ATAXX
    #codificar
    def stringify_move_ataxx(self, x, y, x2, y2):
        return f"MOVE {x},{y},{x2},{y2}"
    #descodificar
    def decode_stringify_move_ataxx(self, move_message):
        pattern = r"MOVE (\d+),(\d+),(\d+),(\d+)"
        match = re.match(pattern, move_message)
        if match:
            x, y, x2, y2 = map(int, match.groups())
            return x, y, x2, y2
        else:
            return None
    
    #GO
    def stringify_move_go(self, x, y):
        return f"MOVE {x},{y}"
    
    def decode_stringify_move_go(self, move_message):
        pattern = r"MOVE (\d+),(\d+)"
        match = re.match(pattern, move_message)
        if match:
            x, y = map(int, match.groups())
            return x, y
        else:
            return None
        
    #conectar servividor
    def connect_to_server(self):
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket.connect((self.host, self.port))

        response = self.client_socket.recv(1024).decode()
        print(f"Server ResponseINIT: {response}")

        #atualizar agente(de acordo com o servidor)
        if "1" in response:
            self.ag=1
        else:
            self.ag=2

        #Obter informações do jogo(de acordo com o servidor) 
        Game = response[-4:] #ex: 'A6x6'
        print("Playing:", Game)
        size = int(Game[1]) #ex: 6
        jogo = Game[0]      #ex: 'A'

        return size, jogo
    
    def get_ag(self):
        return self.ag

##### server

In [9]:
# server
Game = configurations['server_game']#"A6x6" "A5x5" "A4x4" "G7x7" "G9x9" 

def start_server(host=comunication['server_host'], port=comunication['port']):

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)   
    server_socket.bind((host, port))
    server_socket.listen(2)                
    print("Waiting for two agents to connect...")

    agent1, addr1 = server_socket.accept()
    print("Agent 1 connected from", addr1)
    bs=b'AG1 '+Game.encode()
    agent1.sendall(bs)   #destinatário.sendall(mensagem)


    agent2, addr2 = server_socket.accept()
    print("Agent 2 connected from", addr2)
    bs=b'AG2 '+Game.encode()
    agent2.sendall(bs)    


    agents = [agent1, agent2]
    current_agent = 0

    jog=0

    while True:
        try:
            #Recebe e descodifica a mensagem/JOGADA do 1º agente a entrar no server = agents[0]
            #1024 bytes->max espaço de mensagem && .recv()->recebe mensagem
            data = agents[current_agent].recv(1024).decode()
            if not data:
                print("server didn't receive data")
                break

            # Process the move (example: "MOVE X,Y")
            print(current_agent, " -> ",data)
            jog = jog+1
            
            # Send back a response: - VALID or INVALID to the sender, if the move is(n´t) valid
            if is_valid_move(data):
                #Se for válida, envia a jogada para o agente adeversário
                agents[1-current_agent].sendall(data.encode())
                print("server has sent")
            else:
                print("invalid move")
            #agents[current_agent].sendall(b'INVALID')

            # Switch to the other agent
            current_agent = 1-current_agent  #mudar no jogo também?
            time.sleep(1)

        except Exception as e:
            print("Error:", e)
            traceback.print_exc()
            break
    
    print("\n-----------------\nGAME END\n-----------------\n")
    time.sleep(1)
    agent1.close()
    agent2.close()
    server_socket.close()

def is_valid_move(move):
    #we might not need this, so let's keep it true
    return True 

if __name__ == "__main__":
    start_server()

Waiting for two agents to connect...


KeyboardInterrupt: 

# Main

# main

In [ ]:
def main():

    #comunication
    if configurations['comunication']:
        player = Client(host = comunication['client_host'], port = comunication['port'])
        size, jogo = player.connect_to_server()
        ag = player.get_ag()
        _game_mode = game_mode['agent_vs_agent']
    #no comunicatin
    else:
        size = configurations['size']
        jogo = configurations['jogo']
        _game_mode = configurations['game_mode']
        player = None
        ag = None

    if jogo == 'G':
        game = Go(size)
        #7 ou 9?
        initial_state = board7['board'] if size == 7 else board9['board']
        time_between_moves = configurations['time_between_moves']

        interfaceGo(size, game, initial_state, _game_mode, time_between_moves, client = player, ag = ag)

    elif jogo == 'A':
        game = Ataxx(size)
        #6 5 ou 4?
        initial_state = board4['board'] if size == 4 else (board6['board'] if size == 6 else board5['board'])
        time_between_moves = configurations['time_between_moves']

        interfaceAtaxx(size, game, initial_state, _game_mode, time_between_moves, client=player, ag=ag)
       
    
if __name__ == "__main__":
    main()

invalid move
invalid move
invalid move
cursor out of range
PASS
cursor out of range
PASS
cursor out of range
PASS
cursor out of range
PASS
cursor out of range
PASS
cursor out of range
PASS
cursor out of range
PASS
cursor out of range
PASS
____final_counting___

Black captured: 0 White captured: 0

Black points: 0 White points: 44.5

________Winner_______:  white  by 44.5  difference
[[-9 -9 -1 -9 -9 -1 -9]
 [-9 -9 -9 -9 -9 -9 -9]
 [-9 -9 -9 -9 -1 -9 -9]
 [-1 -9 -9 -1  1 -9 -9]
 [-9 -9 -9  1 -9 -9 -9]
 [-1 -9 -9 -9 -9 -9 -1]
 [-9 -9 -9 -9 -9 -9 -1]]
Game Over - Winner: WHITE
cursor out of range
PASS
____final_counting___

Black captured: 0 White captured: 0

Black points: 0 White points: 44.5

________Winner_______:  white  by 44.5  difference
[[-9 -9 -1 -9 -9 -1 -9]
 [-9 -9 -9 -9 -9 -9 -9]
 [-9 -9 -9 -9 -1 -9 -9]
 [-1 -9 -9 -1  1 -9 -9]
 [-9 -9 -9  1 -9 -9 -9]
 [-1 -9 -9 -9 -9 -9 -1]
 [-9 -9 -9 -9 -9 -9 -1]]
Game Over - Winner: WHITE
cursor out of range
PASS
____final_counting___

Blac

now let's to the algotithm

#                Alpha Zero


###  For Ataxx and Go

In [1]:
# import what's needed
import numpy as np
import math
import time
import random
import asyncio

import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)

from tqdm.notebook import trange

In [2]:
class Node:
    def __init__(self, game, args, state, parent=None, action_taken=None, prior=0, visit_count = 0):
        self.game = game
        self.args = args
        self.state = state
        self.parent = parent
        self.action_taken = action_taken
        self.prior = prior 

        self.children = []
        self.visit_count = visit_count
        self.value_sum = 0
    
    def get_last_nine(self):
        #for thre fold repetition rule
        return self.last_nine
    
    def is_fully_expanded(self):
        return len(self.children) > 0

    def select(self):
        best_child = None
        best_ucb = -np.inf 
        #selection by higher UCB
        for child in self.children:
            ucb = self.get_ucb(child)
            if ucb > best_ucb:
                best_child = child
                best_ucb = ucb
        
        return best_child
   
    def get_ucb(self, child):
        if child.visit_count == 0:
            q_value = 0
        else:
            q_value = 1 - ((child.value_sum/child.visit_count)+1) / 2   
        return q_value + self.args['C'] * (math.sqrt(self.visit_count)/(child.visit_count+1)) * child.prior
   
    def expand(self, policy):
        #we exapnd all possible in one expansion
        child = None
        for action, prob in policy.items():
            if prob > 0:
                child_state = self.state.copy()
                child_state = self.game.play_move(action, child_state,1)
                child_state = self.game.change_perspective(child_state)

                child = Node(self.game, self.args, child_state, self, action, prob)
                self.children.append(child)
  
    def backpropagate(self, value):
        self.value_sum += value
        self.visit_count += 1
        value = self.game.get_opponent_value(value)
        if self.parent is not None:
            self.parent.backpropagate(value)

class MCTS:
    def __init__(self, game, args, model):
        self.game = game
        self.args = args
        self.model = model
    
    @torch.no_grad() 
    def search(self, state):
        root = Node(self.game, self.args, state, visit_count=1) 
        
        full_action_space_dict = self.game.action_dict()  

        # Add noise to root node
        policy, _ = self.model(
            torch.tensor(self.game.get_encoded_state(state)).unsqueeze(0)
        )
        policy = torch.softmax(policy, axis=1).squeeze(0).cpu().numpy()
        policy = (1 - self.args['dirichlet_epsilon']) * policy + self.args['dirichlet_epsilon'] \
            * np.random.dirichlet([self.args['dirichlet_alpha']] * self.game.action_size)

        policy /= np.sum(policy)           

        for i in range(len(policy)):
            if policy[i] > 0:
                play_key_at_index = list(full_action_space_dict.keys())[i]   
                if self.game.verify_move(play_key_at_index, state, 1):    
                    full_action_space_dict[play_key_at_index] = policy[i]
                else:
                    full_action_space_dict[play_key_at_index] = 0.0
            else:
                play_at_index = list(full_action_space_dict.keys())[i]    
                full_action_space_dict[play_at_index] = 0.0

        root.expand(full_action_space_dict)

        for search in range(self.args['num_searches']):
            node = root

            # Selection 
            while node.is_fully_expanded():
                node = node.select()

            # Check if the node we selected is terminal
            value, is_terminal = self.game.get_value_and_terminated(node.state, 1) 
            value = self.game.get_opponent_value(value)  
  
            # Expansion
            if not is_terminal:
                policy, value = self.model(
                    torch.tensor(self.game.get_encoded_state(node.state)).unsqueeze(0)  
                )
                policy = torch.softmax(policy, axis=1).squeeze(0).cpu().numpy() 
                # Normalize policy
                policy /= np.sum(policy) 
                # Mask valide plays
                for i in range(len(policy)):
                    if policy[i] > 0:
                        play_key_at_index = list(full_action_space_dict.keys())[i]   
                        if self.game.verify_move(play_key_at_index, node.state, 1):    
                            full_action_space_dict[play_key_at_index] = policy[i]
                        else:
                            full_action_space_dict[play_key_at_index] = 0.0
                    else:
                        play_at_index = list(full_action_space_dict.keys())[i]    
                        full_action_space_dict[play_at_index] = 0.0

                value = value.item() 
         
                node.expand(full_action_space_dict)
       
            node.backpropagate(value) 

        actions_dict = self.game.action_dict()
        for child in root.children:
            action = child.action_taken
            actions_dict[action] = child.visit_count
        action_probs = np.array(list(actions_dict.values()))
        action_probs /= np.sum(action_probs)
        return action_probs

## the Neural Network

In [3]:
class ResNet(nn.Module):
    #Res Blocks are better dealing with gradient vanishing on deeper networks
    def __init__(self, game, num_resBlocks, num_hidden):
        super().__init__()
        self.startBlock = nn.Sequential(
            nn.Conv2d(3, num_hidden, kernel_size=3, padding=1),     
            nn.BatchNorm2d(num_hidden),
            nn.ReLU()
        )

        self.backBone = nn.ModuleList(
            [ResBlock(num_hidden, 3) for i in range(num_resBlocks)]
        )

        self.policyHead = nn.Sequential(
            nn.Conv2d(num_hidden, 32, kernel_size=3, padding=1), 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32 * game.size * game.size, game.action_size)  
        )                                                 
        self.valueHead = nn.Sequential(
            nn.Conv2d(num_hidden, 3, kernel_size=3, padding=1),  
            nn.BatchNorm2d(3), 
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3 * game.size * game.size, 1),  
            nn.Tanh()
        )
        
    def forward(self, x):
        x = self.startBlock(x)
        for resBlock in self.backBone:
            x = resBlock(x)
        policy = self.policyHead(x)
        value = self.valueHead(x)
        return policy, value
    
class ResBlock(nn.Module):
    def __init__(self, num_hidden, board_size):
        super().__init__()
        self.conv1 = nn.Conv2d(num_hidden, num_hidden, kernel_size=3, padding=1)  
        self.bn1 = nn.BatchNorm2d(num_hidden)
        self.conv2 = nn.Conv2d(num_hidden, num_hidden, kernel_size=3, padding=1) 
        self.bn2 = nn.BatchNorm2d(num_hidden)
        
    def forward(self, x):
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        x = F.relu(x)
        return x

## Alpha Zero

In [4]:
class AlphaZero:
    def __init__(self, model, optimizer, game, args):
        self.model = model
        self.optimizer = optimizer
        self.game = game
        self.args = args
        self.mcts = MCTS(game, args, model)
        
    def selfPlay(self):
        memory = []
        player = 1
        state = self.game.make_s0()

        
        while True:          
            if (player == -1):
                neutral_state = self.game.change_perspective(state)
            else:
                neutral_state = state

            action_probs = self.mcts.search(neutral_state)

            memory.append((neutral_state, action_probs, player)) 
            
            temperature_action_probs = action_probs ** (1 / self.args['temperature'])
            temperature_action_probs /= np.sum[temperature_action_probs]
            all_actions = self.game.get_all_actions()
            action_index = np.random.choice(self.game.action_size, p=temperature_action_probs)
            action = all_actions[action_index]
      
            state = self.game.play_move(action, state, player)
         
            value, is_terminal = self.game.get_value_and_terminated(state, player)
            
            if is_terminal or self.game.is_stuck(state, -player):
                returnMemory = []
                for hist_neutral_state, hist_action_probs, hist_player in memory:
                    hist_outcome = value if hist_player == player else self.game.get_opponent_value(value)
                    returnMemory.append((
                        self.game.get_encoded_state(hist_neutral_state),
                        hist_action_probs,
                        hist_outcome
                    ))
                return returnMemory
            
            player = self.game.change_turn(player)
                
    def train(self, memory):
        random.shuffle(memory)
        for batchIdx in range(0, len(memory), self.args['batch_size']):
            sample = memory[batchIdx:min(len(memory) - 1, batchIdx + self.args['batch_size'])] # Change to memory[batchIdx:batchIdx+self.args['batch_size']] in case of an error
            state, policy_targets, value_targets = zip(*sample)
            
            state, policy_targets, value_targets = np.array(state), np.array(policy_targets), np.array(value_targets).reshape(-1, 1)
            
            state = torch.tensor(state, dtype=torch.float32)
            policy_targets = torch.tensor(policy_targets, dtype=torch.float32)
            value_targets = torch.tensor(value_targets, dtype=torch.float32)
            
            out_policy, out_value = self.model(state)
            
            policy_loss = F.cross_entropy(out_policy, policy_targets)
            value_loss = F.mse_loss(out_value, value_targets)
            loss = policy_loss + value_loss
            
            self.optimizer.zero_grad() 
            loss.backward()
            self.optimizer.step() 
    
    def learn(self):
        for iteration in range(self.args['num_iterations']):
            memory = []
            
            self.model.eval()
            for selfPlay_iteration in trange(self.args['num_selfPlay_iterations']):
                memory += self.selfPlay()
                
            self.model.train()
            for epoch in trange(self.args['num_epochs']):
                self.train(memory)
            
            torch.save(self.model.state_dict(), f"model_{iteration}.pt")
            torch.save(self.optimizer.state_dict(), f"optimizer_{iteration}.pt")

In [5]:
## AlphaZero Parallel

In [6]:
class MCTSParallel:
    def __init__(self, game, args, model):
        self.game = game
        self.args = args
        self.model = model
    
        
    @torch.no_grad()
    def search(self, states, spGames):
        full_action_space_dict = self.game.action_dict()  

        # Add noise to root node
        policy, _ = self.model(
            torch.tensor(self.game.get_encoded_state(states))
        )
        policy = torch.softmax(policy, axis=1).cpu().numpy()
        policy = (1 - self.args['dirichlet_epsilon']) * policy + self.args['dirichlet_epsilon'] \
            * np.random.dirichlet([self.args['dirichlet_alpha']] * self.game.action_size, size=policy.shape[0])

        for j, spg in enumerate(spGames):
            spg_policy = policy[j]
            # Normalize Policys
            spg_policy /= np.sum(spg_policy)
            # Mask valide moves
            for i in range(len(spg_policy)):
                if spg_policy[i] > 0:
                    play_key_at_index = list(full_action_space_dict.keys())[i]   
                    if self.game.verify_move(play_key_at_index, states[j], 1):    
                        full_action_space_dict[play_key_at_index] = spg_policy[i]
                    else:
                        full_action_space_dict[play_key_at_index] = 0.0
                else:
                    play_at_index = list(full_action_space_dict.keys())[i]    
                    full_action_space_dict[play_at_index] = 0.0

            spg.root = Node(self.game, self.args, states[j], visit_count=1)
            spg.root.expand(full_action_space_dict) 


        for search in range(self.args['num_searches']):
            for spg in spGames:
                spg.node = None
                node = spg.root

                while node.is_fully_expanded():
                    node = node.select()
            
                value, is_terminal = self.game.get_value_and_terminated(node.state, 1) 
                value = self.game.get_opponent_value(value)

                if is_terminal:
                    node.backpropagate(value)
                else:
                    spg.node = node
                
            expandable_spGames = [mappingIdx for mappingIdx in range(len(spGames)) if spGames[mappingIdx].node is not None]

            if (len(expandable_spGames)) > 0:
                states = np.stack([spGames[mappingIdx].node.state for mappingIdx in expandable_spGames])
                policy, value = self.model(
                    torch.tensor(self.game.get_encoded_state(states)) 
                )
                policy = torch.softmax(policy, axis=1).cpu().numpy()
                value = value.cpu().numpy()

            for j, mappingIdx in enumerate(expandable_spGames):
                node = spGames[mappingIdx].node
                spg_policy, spg_value = policy[j], value[j]
                spg_policy /= np.sum(spg_policy) 
                for i in range(len(spg_policy)):
                    if spg_policy[i] > 0:
                        play_key_at_index = list(full_action_space_dict.keys())[i]   
                        if self.game.verify_move(play_key_at_index, node.state, 1):    
                            full_action_space_dict[play_key_at_index] = spg_policy[i]
                        else:
                            full_action_space_dict[play_key_at_index] = 0.0
                    else:
                        play_at_index = list(full_action_space_dict.keys())[i]    
                        full_action_space_dict[play_at_index] = 0.0

                node.expand(full_action_space_dict)
                node.backpropagate(spg_value) 

        
class AlphaZeroParallel:
    def __init__(self, model, optimizer, game, args):
        self.model = model
        self.optimizer = optimizer
        self.game = game
        self.args = args
        self.mcts = MCTSParallel(game, args, model)
        
    def selfPlay(self):
        return_memory = []
        player = 1
        spGames = [SPG(self.game) for spg in range(self.args['num_parallel_games'])]

        while len(spGames) > 0:
            states = np.stack([spg.state for spg in spGames])

            if (player == -1):
                neutral_states = self.game.change_perspective(states)
            else:
                neutral_states = states
            
            self.mcts.search(neutral_states, spGames)
            
            for i in range(len(spGames))[::-1]:
                spg = spGames[i]

                actions_dict = self.game.action_dict()
                for child in spg.root.children:
                    actions_dict[child.action_taken] = child.visit_count
                action_probs = np.array(list(actions_dict.values()))
                action_probs /= np.sum(action_probs)

                spg.memory.append((spg.root.state, action_probs, player)) 

                temperature_action_probs = action_probs ** (1 / self.args['temperature'])
                temperature_action_probs /= np.sum(temperature_action_probs)
                all_actions = self.game.get_all_actions()
                action_index = np.random.choice(self.game.action_size, p=temperature_action_probs)
                action = all_actions[action_index]

                spg.state = self.game.play_move(action, spg.state, player)
            
                value, is_terminal = self.game.get_value_and_terminated(spg.state, player)
            
                if is_terminal or self.game.is_stuck(spg.state, -player):
                    for hist_neutral_state, hist_action_probs, hist_player in spg.memory:
                        hist_outcome = value if hist_player == player else self.game.get_opponent_value(value)
                        return_memory.append((
                            self.game.get_encoded_state(hist_neutral_state),
                            hist_action_probs,
                            hist_outcome
                        ))
                    del spGames[i]
            
            player = self.game.change_turn(player)
        
        return return_memory
                
    def train(self, memory):
        random.shuffle(memory)
        for batchIdx in range(0, len(memory), self.args['batch_size']):
            sample = memory[batchIdx:min(len(memory) - 1, batchIdx + self.args['batch_size'])] # Change to memory[batchIdx:batchIdx+self.args['batch_size']] in case of an error
            state, policy_targets, value_targets = zip(*sample)
            
            state, policy_targets, value_targets = np.array(state), np.array(policy_targets), np.array(value_targets).reshape(-1, 1)
            
            state = torch.tensor(state, dtype=torch.float32)
            policy_targets = torch.tensor(policy_targets, dtype=torch.float32)
            value_targets = torch.tensor(value_targets, dtype=torch.float32)
            
            out_policy, out_value = self.model(state)
            
            policy_loss = F.cross_entropy(out_policy, policy_targets)
            value_loss = F.mse_loss(out_value, value_targets)
            loss = policy_loss + value_loss
            
            self.optimizer.zero_grad() 
            loss.backward()
            self.optimizer.step() 
    
    def learn(self):
        for iteration in range(self.args['num_iterations']):
            memory = []
            
            self.model.eval()
            for selfPlay_iteration in trange(self.args['num_selfPlay_iterations'] // self.args['num_parallel_games']):
                memory += self.selfPlay()
                
            self.model.train()
            for epoch in trange(self.args['num_epochs']):
                self.train(memory)
            
            torch.save(self.model.state_dict(), f"model_{iteration}.pt")
            torch.save(self.optimizer.state_dict(), f"optimizer_{iteration}.pt")


class SPG:
    def __init__(self, game):
        self.state = game.make_s0()
        self.memory = []
        self.root = None
        self.node = None

In [7]:
### train

In [8]:
#this are test values
args = {
    'C': 2,
    'num_searches': 5,#60,
    'num_iterations': 3,
    'num_selfPlay_iterations': 20, #500,
    'num_parallel_games': 10, #100,
    'num_epochs': 2, #4
    'batch_size': 32, #64
    'temperature': 1.25, 
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.3

}

In [10]:
ataxx = Ataxx(4)

model = ResNet(ataxx, 10, 256)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.0001)

alphaZero = AlphaZeroParallel(model, optimizer, ataxx, args)
alphaZero.learn()

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]